In [1]:
!pip install -q -U transformers==4.53.3 num2words #nuscenes-devkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 kB 9.3 MB/s eta 0:00:00


# Load the Model

In [3]:
import torch
from transformers import AutoProcessor, AutoModelForImageTextToText
from PIL import Image
import num2words

model_path = "HuggingFaceTB/SmolVLM2-2.2B-Instruct"

processor = AutoProcessor.from_pretrained(model_path)
model = AutoModelForImageTextToText.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    device_map="cuda",
)

for param in model.parameters():
    param.requires_grad = False

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.03G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

In [9]:
processor.image_processor.max_image_size["longest_edge"]= 384
processor.image_processor.do_image_splitting=False
processor.image_processor.do_resize=True

In [5]:
teacher = model.model.vision_model

# dataset

In [6]:
from datasets import load_dataset

dataset = load_dataset("MehdiJmlkh/nuscenes")

README.md:   0%|          | 0.00/703 [00:00<?, ?B/s]

train-00000-of-00006.parquet:   0%|          | 0.00/454M [00:00<?, ?B/s]

train-00001-of-00006.parquet:   0%|          | 0.00/462M [00:00<?, ?B/s]

train-00002-of-00006.parquet:   0%|          | 0.00/465M [00:00<?, ?B/s]

train-00003-of-00006.parquet:   0%|          | 0.00/456M [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/369M [00:00<?, ?B/s]

eval-00000-of-00001.parquet:   0%|          | 0.00/355M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3241 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/422 [00:00<?, ? examples/s]

Generating eval split:   0%|          | 0/411 [00:00<?, ? examples/s]

In [ ]:
drive_lm = load_dataset("MehdiJmlkh/DriveLM")

README.md:   0%|          | 0.00/718 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.00M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/257k [00:00<?, ?B/s]

eval-00000-of-00001.parquet:   0%|          | 0.00/246k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/106338 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/13292 [00:00<?, ? examples/s]

Generating eval split:   0%|          | 0/13293 [00:00<?, ? examples/s]

# get the labels

In [ ]:
dataset['train'][0]

{'CAM_FRONT_LEFT': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1600x900>,
 'CAM_FRONT': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1600x900>,
 'CAM_FRONT_RIGHT': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1600x900>,
 'CAM_BACK_LEFT': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1600x900>,
 'CAM_BACK': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1600x900>,
 'CAM_BACK_RIGHT': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1600x900>}

In [ ]:
drive_lm['train'][510]

{'scene_token': '1977a1c98a6c4eb79fbc2a6dc0da9b0f',
 'frame_token': '947c1b70cff6486b87903d9665064ad8',
 'scene_description': 'The ego vehicle is parked on the right side of the current road.',
 'task': 'perception',
 'question': 'Are there moving trailers to the front of the ego car?',
 'answer': 'Yes.',
 'nuscenes_index': 13}

In [ ]:
index_to_nusc = {}
for example in drive_lm['train']:
    index_to_nusc[example['nuscenes_index']] = (example['scene_token'], example['frame_token'])

In [13]:
import torch
from datasets import Dataset
from PIL import Image

CAM_KEYS = [
    "CAM_BACK", "CAM_BACK_LEFT", "CAM_BACK_RIGHT",
    "CAM_FRONT", "CAM_FRONT_LEFT", "CAM_FRONT_RIGHT"
]

processed_samples = []

for i in range(3):#len(dataset['train'])):
    for cam in CAM_KEYS:
        image = dataset['train'][i][cam]
        pixel_values = torch.tensor(processor.image_processor(image).pixel_values).view(1,3,384,384).to(device="cuda", dtype=torch.bfloat16)
        # print(pixel_values.shape)
        with torch.no_grad():
            embedding = teacher(pixel_values).last_hidden_state

        processed_samples.append({
            "nusc_idx": i,
            "position": cam,
            "data": pixel_values,
            "label": embedding,
            "image": image,
        })

torch.Size([1, 3, 384, 384])
torch.Size([1, 3, 384, 384])
torch.Size([1, 3, 384, 384])
torch.Size([1, 3, 384, 384])
torch.Size([1, 3, 384, 384])
torch.Size([1, 3, 384, 384])
torch.Size([1, 3, 384, 384])
torch.Size([1, 3, 384, 384])
torch.Size([1, 3, 384, 384])
torch.Size([1, 3, 384, 384])
torch.Size([1, 3, 384, 384])
torch.Size([1, 3, 384, 384])
torch.Size([1, 3, 384, 384])
torch.Size([1, 3, 384, 384])
torch.Size([1, 3, 384, 384])
torch.Size([1, 3, 384, 384])
torch.Size([1, 3, 384, 384])
torch.Size([1, 3, 384, 384])


In [19]:
import huggingface_hub
huggingface_hub.login()

In [20]:
hf_dataset = Dataset.from_list(processed_samples)

hf_dataset.push_to_hub("ArianFiroozi/Nuscenes-Knowledge-Transfer")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/ArianFiroozi/Nuscenes-Knowledge-Transfer/commit/8dfb393e2acd2c627c5c7b95da7da8519c9e9638', commit_message='Upload dataset', commit_description='', oid='8dfb393e2acd2c627c5c7b95da7da8519c9e9638', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/ArianFiroozi/Nuscenes-Knowledge-Transfer', endpoint='https://huggingface.co', repo_type='dataset', repo_id='ArianFiroozi/Nuscenes-Knowledge-Transfer'), pr_revision=None, pr_num=None)

In [22]:
from datasets import load_dataset

loaded_dataset = load_dataset("ArianFiroozi/Nuscenes-Knowledge-Transfer")
display(loaded_dataset)

README.md:   0%|          | 0.00/487 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/18 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['nusc_idx', 'position', 'data', 'label', 'image'],
        num_rows: 18
    })
})

In [26]:
torch.tensor(loaded_dataset['train'][0]['data']).shape

torch.Size([1, 3, 384, 384])